In [7]:
import pandas as pd
import numpy as np

## MRP

In [8]:
# initial assumptions
weeks = 12
assumptions = pd.DataFrame({
    'LS' : [82050, 58266, 49723],
    'LT' : [1, 2, 3],
    'int_inventory' : [10000, 10000, 10000]
})
assumptions.index = ["Japan", "Germany", "USA"]

scheduled_receipts = {
    "Japan": [10000, 10000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "Germany": [10000, 10000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "USA": [10000, 0, 0, 0, 0, 0, 0, 10000, 0, 0, 0, 0],
}

customer_orders = {
    "Japan": [0, 82050 , 0, 0, 82050, 0, 82050, 0, 0, 0, 0, 0],
    "Germany": [58266, 0, 116532, 0, 0, 116532, 0, 0, 116532, 0, 0, 0],
    "USA": [49723, 49723, 0, 99446, 0, 49723, 0, 99446, 0, 49723, 0, 0],
}

In [9]:
assumptions

,LS,LT,int_inventory
Japan,82050,1,10000
Germany,58266,2,10000
USA,49723,3,10000


In [10]:
def MRP(Country, LS, int_inventory, orders, LT, scheduled_receipts, weeks=12):
    gross_requirements = orders
    POH = [0] * weeks
    net_requirements = [0] * weeks
    planned_order_releases = [0] * weeks
    
    # Starting point for POH
    POH[0] = int_inventory + scheduled_receipts[Country][0] - gross_requirements[0]

    for t in range(weeks):
        # Update POH for each week
        if t > 0:
            POH[t] = POH[t - 1] + scheduled_receipts[Country][t] - gross_requirements[t]
        
        # If POH is negative, calculate net requirements and release orders
        if POH[t] < 0:
            net_requirements[t] = np.abs(POH[t])  # Calculate how much is needed
            release_week = max(0, t - LT)  # Determine the release week based on lead time
            planned_order_releases[release_week] = (round(net_requirements[t] / LS)) * LS  # Place order in lot size
            
            # Update POH with planned order release after orders are placed
            if release_week > 0:  # Ensure we update POH after placing the order
                POH[release_week] += planned_order_releases[release_week]
            else:
                POH[t] += planned_order_releases[release_week]
        
        # Update POH after planned order releases
        if t > 0 and POH[t] < 0:
            POH[t] = max(0, POH[t])  # Ensure POH does not go negative

    # Creating DataFrame for the MRP table
    df = pd.DataFrame({
        "Gross Requirements": gross_requirements,
        "Scheduled Receipts": scheduled_receipts[Country],
        "Projected On Hand": POH,
        "Net Requirements": net_requirements,
        "Planned Order Releases": planned_order_releases,
    }, index=[f"Week {i + 1}" for i in range(weeks)])

    return df.T

In [11]:
for i in ['Japan', 'Germany', 'USA']:
    print(f'MRP table for {i}')
    print('----------------------------------------------------------------------------------')
    df = MRP(
        i,
        assumptions.loc[i]['LS'],
        assumptions.loc[i]['int_inventory'],
        customer_orders[i],
        assumptions.loc[i]['LT'],
        scheduled_receipts,
    )
    print(df, '\n')

MRP table for Japan
----------------------------------------------------------------------------------
                        Week 1  Week 2  Week 3  Week 4  Week 5  Week 6  \
Gross Requirements           0   82050       0       0   82050       0   
Scheduled Receipts       10000   10000       0       0       0       0   
Projected On Hand        20000   30000   30000  112050       0   82050   
Net Requirements             0   52050       0       0   52050       0   
Planned Order Releases   82050       0       0   82050       0   82050   

                        Week 7  Week 8  Week 9  Week 10  Week 11  Week 12  
Gross Requirements       82050       0       0        0        0        0  
Scheduled Receipts           0       0       0        0        0        0  
Projected On Hand            0       0       0        0        0        0  
Net Requirements         82050       0       0        0        0        0  
Planned Order Releases       0       0       0        0        0        